In [1]:
from numpy.fft import fft
import numpy as np
import os
import random
import matplotlib.pyplot as plt

In [2]:
N_data = 2500  # Number of training data in final augmented dataset
L = 1024      # Length of IQ sequence in each sample
D = 100 # Decimation factor for long-term data
SNRdBmin = -10
SNRdBmax = 20

In [ ]:
max_pw  = 10000 # Pulse Width (us)
max_pri = 10000 # Pulse Repetition Interval (us)

In [ ]:
# Select Radar emitter parameters
pw = random.uniform(0, max_pw)
pri = random.uniform(0, max_pri)
pulse_phase = random.uniform(0, 1) # Phase between 0 and 1 as percent of cycle elapsed at start of waveform



